In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.1


In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_relu'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):                
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.6328, train_loss: 2.17023849, train_accuracy: 0.3750, test_accuracy: 0.2355
Epoch: [ 0] [    1/  468] time: 0.8188, train_loss: 2.13479424, train_accuracy: 0.4062, test_accuracy: 0.3367
Epoch: [ 0] [    2/  468] time: 1.0008, train_loss: 2.04457664, train_accuracy: 0.6016, test_accuracy: 0.4629
Epoch: [ 0] [    3/  468] time: 1.4028, train_loss: 2.00121689, train_accuracy: 0.5078, test_accuracy: 0.5216
Epoch: [ 0] [    4/  468] time: 1.5978, train_loss: 1.90660954, train_accuracy: 0.6875, test_accuracy: 0.5890
Epoch: [ 0] [    5/  468] time: 1.8048, train_loss: 1.84356201, train_accuracy: 0.6328, test_accuracy: 0.6248
Epoch: [ 0] [    6/  468] time: 2.0048, train_loss: 1.69335866, train_accuracy: 0.6953, test_accuracy: 0.6345
Epoch: [ 0] [    7/  468] time: 2.1938, train_loss: 1.62527490, train_accuracy: 0.7031, test_accuracy: 0.6516
Epoch: [ 0] [    8/  468] time: 2.3818,

Epoch: [ 0] [   75/  468] time: 12.6444, train_loss: 0.35038292, train_accuracy: 0.9219, test_accuracy: 0.9075
Epoch: [ 0] [   76/  468] time: 12.7938, train_loss: 0.31059837, train_accuracy: 0.8984, test_accuracy: 0.9050
Epoch: [ 0] [   77/  468] time: 12.9408, train_loss: 0.23194771, train_accuracy: 0.9609, test_accuracy: 0.9037
Epoch: [ 0] [   78/  468] time: 13.0858, train_loss: 0.23284449, train_accuracy: 0.9531, test_accuracy: 0.9091
Epoch: [ 0] [   79/  468] time: 13.2288, train_loss: 0.29844299, train_accuracy: 0.8906, test_accuracy: 0.9117
Epoch: [ 0] [   80/  468] time: 13.3778, train_loss: 0.36365953, train_accuracy: 0.8984, test_accuracy: 0.9121
Epoch: [ 0] [   81/  468] time: 13.5434, train_loss: 0.35259894, train_accuracy: 0.9141, test_accuracy: 0.9135
Epoch: [ 0] [   82/  468] time: 13.6934, train_loss: 0.31051612, train_accuracy: 0.8984, test_accuracy: 0.9147
Epoch: [ 0] [   83/  468] time: 13.8364, train_loss: 0.31277061, train_accuracy: 0.8750, test_accuracy: 0.9166
E

Epoch: [ 0] [  149/  468] time: 23.6240, train_loss: 0.24950047, train_accuracy: 0.9219, test_accuracy: 0.9273
Epoch: [ 0] [  150/  468] time: 23.7799, train_loss: 0.19225374, train_accuracy: 0.9375, test_accuracy: 0.9286
Epoch: [ 0] [  151/  468] time: 23.9292, train_loss: 0.16996202, train_accuracy: 0.9297, test_accuracy: 0.9291
Epoch: [ 0] [  152/  468] time: 24.0732, train_loss: 0.25184175, train_accuracy: 0.8906, test_accuracy: 0.9293
Epoch: [ 0] [  153/  468] time: 24.2342, train_loss: 0.31027311, train_accuracy: 0.9375, test_accuracy: 0.9319
Epoch: [ 0] [  154/  468] time: 24.3732, train_loss: 0.25122592, train_accuracy: 0.9297, test_accuracy: 0.9312
Epoch: [ 0] [  155/  468] time: 24.5242, train_loss: 0.25692481, train_accuracy: 0.9297, test_accuracy: 0.9307
Epoch: [ 0] [  156/  468] time: 24.6711, train_loss: 0.30505979, train_accuracy: 0.9297, test_accuracy: 0.9310
Epoch: [ 0] [  157/  468] time: 24.8262, train_loss: 0.33313769, train_accuracy: 0.8906, test_accuracy: 0.9330
E

Epoch: [ 0] [  223/  468] time: 35.6311, train_loss: 0.21544564, train_accuracy: 0.9297, test_accuracy: 0.9394
Epoch: [ 0] [  224/  468] time: 35.7921, train_loss: 0.07750258, train_accuracy: 0.9922, test_accuracy: 0.9390
Epoch: [ 0] [  225/  468] time: 35.9630, train_loss: 0.17434120, train_accuracy: 0.9453, test_accuracy: 0.9346
Epoch: [ 0] [  226/  468] time: 36.1200, train_loss: 0.16275305, train_accuracy: 0.9375, test_accuracy: 0.9322
Epoch: [ 0] [  227/  468] time: 36.2815, train_loss: 0.22031417, train_accuracy: 0.9453, test_accuracy: 0.9309
Epoch: [ 0] [  228/  468] time: 36.4225, train_loss: 0.17791589, train_accuracy: 0.9531, test_accuracy: 0.9323
Epoch: [ 0] [  229/  468] time: 36.5655, train_loss: 0.14168756, train_accuracy: 0.9688, test_accuracy: 0.9340
Epoch: [ 0] [  230/  468] time: 36.7737, train_loss: 0.12538184, train_accuracy: 0.9766, test_accuracy: 0.9359
Epoch: [ 0] [  231/  468] time: 36.9587, train_loss: 0.27681580, train_accuracy: 0.9219, test_accuracy: 0.9389
E

Epoch: [ 0] [  298/  468] time: 49.3727, train_loss: 0.14741725, train_accuracy: 0.9453, test_accuracy: 0.9506
Epoch: [ 0] [  299/  468] time: 49.5267, train_loss: 0.13178770, train_accuracy: 0.9688, test_accuracy: 0.9519
Epoch: [ 0] [  300/  468] time: 49.7021, train_loss: 0.18091258, train_accuracy: 0.9297, test_accuracy: 0.9526
Epoch: [ 0] [  301/  468] time: 49.8881, train_loss: 0.20601708, train_accuracy: 0.9219, test_accuracy: 0.9515
Epoch: [ 0] [  302/  468] time: 50.0681, train_loss: 0.16505370, train_accuracy: 0.9453, test_accuracy: 0.9501
Epoch: [ 0] [  303/  468] time: 50.2431, train_loss: 0.15335521, train_accuracy: 0.9609, test_accuracy: 0.9505
Epoch: [ 0] [  304/  468] time: 50.4593, train_loss: 0.22748503, train_accuracy: 0.9297, test_accuracy: 0.9512
Epoch: [ 0] [  305/  468] time: 50.6523, train_loss: 0.19427006, train_accuracy: 0.9609, test_accuracy: 0.9520
Epoch: [ 0] [  306/  468] time: 50.8013, train_loss: 0.06552469, train_accuracy: 0.9766, test_accuracy: 0.9526
E

Epoch: [ 0] [  373/  468] time: 62.6221, train_loss: 0.12495139, train_accuracy: 0.9531, test_accuracy: 0.9568
Epoch: [ 0] [  374/  468] time: 62.8411, train_loss: 0.17868739, train_accuracy: 0.9609, test_accuracy: 0.9573
Epoch: [ 0] [  375/  468] time: 63.0121, train_loss: 0.10244502, train_accuracy: 0.9609, test_accuracy: 0.9570
Epoch: [ 0] [  376/  468] time: 63.1551, train_loss: 0.23818237, train_accuracy: 0.9219, test_accuracy: 0.9575
Epoch: [ 0] [  377/  468] time: 63.3094, train_loss: 0.19082063, train_accuracy: 0.9375, test_accuracy: 0.9578
Epoch: [ 0] [  378/  468] time: 63.4560, train_loss: 0.12318152, train_accuracy: 0.9688, test_accuracy: 0.9576
Epoch: [ 0] [  379/  468] time: 63.5976, train_loss: 0.11355478, train_accuracy: 0.9609, test_accuracy: 0.9576
Epoch: [ 0] [  380/  468] time: 63.7556, train_loss: 0.09438721, train_accuracy: 0.9844, test_accuracy: 0.9569
Epoch: [ 0] [  381/  468] time: 63.9942, train_loss: 0.27398556, train_accuracy: 0.9141, test_accuracy: 0.9562
E

Epoch: [ 0] [  448/  468] time: 75.9879, train_loss: 0.13798593, train_accuracy: 0.9531, test_accuracy: 0.9576
Epoch: [ 0] [  449/  468] time: 76.1699, train_loss: 0.09722297, train_accuracy: 0.9609, test_accuracy: 0.9569
Epoch: [ 0] [  450/  468] time: 76.3281, train_loss: 0.07081491, train_accuracy: 0.9766, test_accuracy: 0.9551
Epoch: [ 0] [  451/  468] time: 76.4831, train_loss: 0.11989032, train_accuracy: 0.9609, test_accuracy: 0.9539
Epoch: [ 0] [  452/  468] time: 76.7061, train_loss: 0.20490278, train_accuracy: 0.9531, test_accuracy: 0.9564
Epoch: [ 0] [  453/  468] time: 76.8811, train_loss: 0.10995957, train_accuracy: 0.9688, test_accuracy: 0.9592
Epoch: [ 0] [  454/  468] time: 77.0403, train_loss: 0.12804887, train_accuracy: 0.9609, test_accuracy: 0.9593
Epoch: [ 0] [  455/  468] time: 77.2593, train_loss: 0.09920587, train_accuracy: 0.9688, test_accuracy: 0.9578
Epoch: [ 0] [  456/  468] time: 77.4393, train_loss: 0.06835880, train_accuracy: 0.9922, test_accuracy: 0.9536
E